In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [164]:
import pandas as pd
import random
import string
import nltk

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
from navec import Navec
from slovnet.model.emb import NavecEmbedding

from tqdm.notebook import tqdm
tqdm.pandas()

# Подгружаем данные

In [ ]:
glossary = pd.read_csv('gdrive/MyDrive/AI_HUB_HACK/gloss.csv')
glossary

,Сокращение,Расшифровка
0,лк,личный кабинет
1,БиР,Беременность и роды
2,зп,заработная плата
3,НДФЛ,Налог на доходы физических лиц
4,СТД,срочный трудовой договор
5,ТК,трудовой договор
6,АО,авансовый отчет
7,SLA,сроки
8,ЭЦП,электронная цифровая подпись
9,КР,кадровый резерв


In [ ]:
popular_phrases = pd.read_csv('gdrive/MyDrive/AI_HUB_HACK/popular_phrases.csv')
popular_phrases

,Ответ,Вопрос
0,Для оформления обращения в техническую поддерж...,выйти
1,Чат-бот находится в стадии пилотирования и обу...,Уберите этот чат бот он очень мешает
2,Чат-бот находится в стадии пилотирования и обу...,вернуть старый дизайн
3,Для оформления обращения в техническую поддерж...,хочу создать заявку
4,Для оформления обращения в техническую поддерж...,мне нужно создать заявку
5,Чат-бот находится в стадии пилотирования и обу...,бот тупой
6,Чат-бот находится в стадии пилотирования и обу...,бот не помогает
7,Чат-бот находится в стадии пилотирования и обу...,как убрать бота
8,Для оформления обращения в техническую поддерж...,мне сказали создать заявку в поддержке
9,Для оформления обращения в техническую поддерж...,выход


In [ ]:
qa_df=pd.read_csv('gdrive/MyDrive/AI_HUB_HACK/qa.csv')
qa_df.drop('Unnamed: 0', axis=1, inplace=True)
qa_df.head(5)

,question,content,category
0,"Я сменил автомобить, на учет еще не поставил, ...",Для внесения данных по личному автомобилю обра...,автомобиль
1,Не отображается автомобиль в личном кабинете.,Для внесения данных по личному автомобилю обра...,автомобиль
2,добавить автомобиль,Для внесения данных по личному автомобилю обра...,автомобиль
3,хочу внести данные об автомобиле,Для внесения данных по личному автомобилю обра...,автомобиль
4,Как внести данные об автомобиле?,Для внесения данных по личному автомобилю обра...,автомобиль


# Смотрим на данные

In [ ]:
proportions = qa_df['category'].value_counts(dropna=False).reset_index()
print(proportions.head(5), '\n')
print(proportions.tail(10))

           category  count
0                ЛК    481
1         поддержка    276
2            табель    168
3            отпуск    119
4  удаленная работа     98 

         category  count
22       обучение      7
23   командировка      5
24       оператор      5
25            дмс      3
26        перевод      2
27             СБ      1
28            МЧД      1
29            SED      1
30  выручай-карта      1
31         Отпуск      1


In [ ]:
id_map_content = {}
for x in qa_df['content']:
  id_map_content[x] = id_map_content.get(x, len(id_map_content))
qa_df['content_id'] = qa_df['content'].map(id_map_content)

In [ ]:
id_map_categories = {}
for x in qa_df['category']:
  id_map_categories[x] = id_map_categories.get(x, len(id_map_categories))
qa_df['category_id'] = qa_df['category'].map(id_map_categories)

In [ ]:
sum(qa_df['content_id'].value_counts() <= 3) / qa_df['content_id'].nunique()

0.6238938053097345

In [ ]:
qa_df['content_id']

,content_id
0,0
1,0
2,0
3,0
4,0
...,...
1671,223
1672,223
1673,225
1674,83


# Один ответ на одну категорию?

In [ ]:
qa_df.sample(5)

,question,content,category,content_id,category_id
613,у сотрудника нет логин и пароль для доступа в ЛК,"При проблемах со входом в личный кабинет, преж...",ЛК,70,8
121,в какую дату происходит удеражание моего доход...,"Обращаем внимание, что работодатель производит...",зарплата,42,5
197,"мне нужно оформить командировку, как это сделать?",В рамках нового сервисного подхода сотрудники ...,командировка,59,7
1402,В какие сроки можно уволиться?,"Согласно ст. 80 ТК РФ, работник имеет право ра...",увольнение,167,23
209,нет отображаются заявки в личном кабинете на п...,"Первым делом, просьба очистить кэш/куки браузе...",ЛК,64,8


In [ ]:
content_cat_dict = {}

for _, row in qa_df.iterrows():
    # Проверяем, есть ли content_id в словаре
    if row['content_id'] not in content_cat_dict:
        # Инициализируем список с первым кортежем (category_id, question)
        content_cat_dict[row['content_id']] = [(row['category_id'], row['question'])]
    else:
        # Получаем список текущих категорий для данного content_id
        existing_categories = {cat_id for cat_id, _ in content_cat_dict[row['content_id']]}

        # Проверяем, есть ли category_id в текущих категориях
        if row['category_id'] not in existing_categories:
            # Добавляем новый кортеж (category_id, question)
            content_cat_dict[row['content_id']].append((row['category_id'], row['question']))


In [ ]:
def get_key_by_value(mapper, value):
  return list(mapper.keys())[list(mapper.values()).index(value)]

count = 1
for content_id, list_cat_id_and_tuple in content_cat_dict.items():
  if len(list_cat_id_and_tuple) > 1:
    print(f'{count}) {get_key_by_value(id_map_content, content_id)} \n')
    for cat_id_and_tuple in list_cat_id_and_tuple:
      print(f'    - {get_key_by_value(id_map_categories, cat_id_and_tuple[0])};  {cat_id_and_tuple[1]} \n')
    count += 1

1) Просьба очистить кеш/куки с использованием клавиш Ctrl + Shift + Delete. При появлении окошка с настройками в разделе "За период" выбрать "За все время". Далее выйти с личного кабинета с использованием клавиши "Выход" и авторизоваться повторно введя логин и пароль. 

    - ЛК;  БЕЛЫЙ ЭКРАН 

    - поддержка;  В мой ЛК поступило 2 задачи на перенос отпуска сотрудниками. При попытке выпонения задачи, выходит ошибка 404. 

    - моя карьера;  Нет доступа к курсу НПС 

2) Кнопка "изменить номер" телефона находится в личном разделе в ЛК. Если доступа к ЛК нет, для смены номера телефона, обратитесь в поддержку 

    - ЛК;  как изменить номер телефона 

    - поддержка;  Проблема в том,что сотрудник не может зайти в ЛК ,тк пароль от ЛК отправлен на другой (неверный номер) 

3) По данному вопросу Вы можете обратиться в кадровую службу, создав заявку "Консультация по HR вопросам" 

    - ЛК;  не верно указан статус в лк у сотрудника . статус уволен, но у сотрудника повторный прием. 

    - п

# Функции для аугменации

In [ ]:
def add_or_remove_punctuation(text):
    """Добавление или удаление знаков препинания."""
    # Возможные варианты добавления знаков препинания
    punctuations = [',', '.', '!', '?']
    words = text.split()

    # Добавляем или удаляем знаки препинания
    if random.random() < 0.5:
        # Добавить знак препинания
        position = random.randint(0, len(words) - 1)
        punct = random.choice(punctuations)
        words[position] = words[position] + punct
    else:
        # Удалить знак препинания, если он есть
        text = text.translate(str.maketrans('', '', string.punctuation))
        words = text.split()

    return ' '.join(words)

In [ ]:
def introduce_typo(text):
    """Создание опечаток в тексте."""
    if not text:
        return text

    words = text.split()
    index = random.randint(0, len(words) - 1)
    word = words[index]

    # Опечатки: замена, пропуск или дублирование символов
    typo_type = random.choice(['swap', 'remove', 'duplicate'])

    if typo_type == 'swap' and len(word) > 1:
        # Меняем местами соседние буквы
        pos = random.randint(0, len(word) - 2)
        word = list(word)
        word[pos], word[pos + 1] = word[pos + 1], word[pos]
        words[index] = ''.join(word)

    elif typo_type == 'remove' and len(word) > 1:
        # Удаляем случайную букву
        pos = random.randint(0, len(word) - 1)
        words[index] = word[:pos] + word[pos + 1:]

    elif typo_type == 'duplicate':
        # Дублируем случайную букву
        pos = random.randint(0, len(word) - 1)
        words[index] = word[:pos] + word[pos] + word[pos:]

    return ' '.join(words)

In [ ]:
def shuffle_words(text):
    """Перестановка порядка слов."""
    words = text.split()
    if len(words) > 1:
        random.shuffle(words)
    return ' '.join(words)

# Пример аугментации

In [ ]:
# Пример использования
text = "всем привет, динозавры"
augmented_text = add_or_remove_punctuation(text)
augmented_text = introduce_typo(augmented_text)
augmented_text = shuffle_words(augmented_text)

print("Оригинальное предложение:", text)
print("Аугментированное предложение:", augmented_text)

Оригинальное предложение: всем привет, динозавры
Аугментированное предложение: привет, всем? динозаввры


# Расширение датасета

In [193]:
AUG_NUM = 30

def balance_dataset(qa_df):
    # Шаг 1: Найти самый частовстречаемый ответ
    max_count = qa_df['content_id'].value_counts().max()

    # Шаг 2: Сбалансировать выборку ответов
    augmented_data = []

    for content_id, group in tqdm(qa_df.groupby('content_id')):
        count = len(group)
        augmented_data.extend(group.to_dict('records'))  # Добавляем все исходные строки

        # Если ответ встречается реже, чем самый частовстречаемый, создаем аугментированные копии вопросов
        for _ in range(min(AUG_NUM, max_count - count)):
            row = group.sample(1).iloc[0].to_dict()  # Случайный вопрос из группы
            question = row['question']

            # Применяем несколько аугментаций последовательно
            augmented_question = add_or_remove_punctuation(question)
            augmented_question = introduce_typo(augmented_question)
            augmented_question = shuffle_words(augmented_question)

            # Сохраняем аугментированный вопрос с исходным ответом
            new_row = row.copy()
            new_row['question'] = augmented_question
            augmented_data.append(new_row)

    # Шаг 3: Создать новый сбалансированный датафрейм
    balanced_qa_df = pd.DataFrame(augmented_data)
    return balanced_qa_df

balanced_qa_df = balance_dataset(qa_df)

  0%|          | 0/226 [00:00<?, ?it/s]

# Navec

In [ ]:
! pip install navec

In [ ]:
! pip install slovnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 1.9 MB/s eta 0:00:00


In [204]:
path = '/content/navec_hudlit_v1_12B_500K_300d_100q.tar'  # 51MB
navec = Navec.load(path)  # ~1 sec, ~100MB RAM

In [205]:
emb = NavecEmbedding(navec)

# Функция для преобразования текста в индексы
def text_to_indices(text, navec):
    tokens = text.split()  # Используем пробелы для токенизации
    indices = [navec.vocab.get(token, navec.vocab['<unk>']) for token in tokens]
    return indices

# Функция для получения эмбеддингов
def get_embeddings(text, emb, navec):
    indices = text_to_indices(text, navec)
    if not indices:
        return torch.zeros(1, emb.embedding_dim)  # Возвращаем нулевой тензор для пустых запросов
    input_tensor = torch.tensor(indices)
    embeddings = emb(input_tensor).mean(dim=0)  # Среднее значение по всем токенам
    return embeddings

# Применение функции к столбцу questions и сохранение результатов
balanced_qa_df['embeddings'] = balanced_qa_df['question'].progress_apply(lambda x: get_embeddings(x, emb, navec).tolist())

# # Сохранение датафрейма с эмбеддингами
# balanced_qa_df.to_csv('data_with_embeddings.csv', index=False)


  0%|          | 0/8416 [00:00<?, ?it/s]

In [206]:
# Преобразование столбца эмбеддингов из списка в numpy массив
balanced_qa_df['embeddings'] = balanced_qa_df['embeddings'].apply(np.array)

In [207]:
# Подготовка данных
X = np.stack(balanced_qa_df['embeddings'].values)  # Преобразуем список массивов в один 2D массив
y = balanced_qa_df['content_id']  # Таргет-классы

In [208]:
# Разделение на тренировочную, валидационную и тестовую выборки (75% / 15% / 10%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25,
                                                    stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.4,
                                                stratify=y_temp, random_state=42)

In [209]:
# Обучение модели логистической регрессии
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [212]:
# Предсказания на тестовой выборке
y_pred = model.predict(X_test)

# Подсчет метрик
accuracy = accuracy_score(y_test, y_pred)
precision_macro = precision_score(y_test, y_pred, average='macro')
precision_micro = precision_score(y_test, y_pred, average='micro')
recall_macro = recall_score(y_test, y_pred, average='macro')
recall_micro = recall_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_micro = f1_score(y_test, y_pred, average='micro')

# Вывод метрик
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: (macro:{precision_macro:.4f}, micro:{precision_micro:.4f})')
print(f'Recall: (macro:{recall_macro:.4f}, micro:{recall_micro:.4f})')
print(f'F1 Score: (macro:{f1_macro:.4f}, micro:{f1_micro:.4f}) \n')


Accuracy: 0.8325
Precision: (macro:0.9057, micro:0.8325)
Recall: (macro:0.8568, micro:0.8325)
F1 Score: (macro:0.8639, micro:0.8325) 



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
